In [217]:
import pandas as pd
import logging

In [218]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [219]:
vote_file = pd.read_csv('brexit_full_updated3.csv')
vote_file

,Constituency ID,full name,PANO,Constituency Name,Party abbreviation,conShare,conShare2,conShare3,bill161,leave,party_old,ref,ref_dummy,party,old_party
0,E14000530,Gerald Howarth,7,Aldershot,Con,50.59,73.403947,73.403947,1.0,1.0,1.0,0.578978,1,1.0,1.0
1,E14000531,Wendy Morton,8,Aldridge-Brownhills,Con,52.05,69.940876,69.940876,1.0,0.0,1.0,0.677963,1,0.0,0.0
2,E14000532,Graham Brady,9,Altrincham and Sale West,Con,52.99,66.503514,66.503514,1.0,1.0,1.0,0.385878,0,1.0,1.0
3,E14000533,Nigel Mills,11,Amber Valley,Con,43.98,55.840528,55.840528,1.0,1.0,1.0,0.652991,1,1.0,1.0
4,E14000534,Nick Herbert,18,Arundel and South Downs,Con,60.79,84.442284,84.442284,NaN,0.0,1.0,0.497011,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,W07000076,Wayne David,114,Caerphilly,Lab,16.59,27.223499,27.223499,1.0,0.0,0.0,0.551360,1,0.0,NaN
656,W07000077,Chris Evans,336,Islwyn,Lab,15.16,23.639482,23.639482,1.0,0.0,0.0,0.589399,1,0.0,NaN
657,W07000078,Alun Cairns,589,Vale Of Glamorgan,Con,46.02,58.527280,58.527280,1.0,0.0,1.0,0.525507,1,0.0,0.0
658,W07000079,Kevin Brennan,129,Cardiff West,Lab,25.15,38.221884,38.221884,0.0,0.0,0.0,0.438226,0,0.0,NaN


In [220]:
politicians_party = {}
politicians_ref = {}
politicians = vote_file['full name'].unique()
for index, row in vote_file.iterrows():
    politicians_party[row['full name']] = row['party']
    politicians_ref[row['full name']] = row['ref_dummy']

In [223]:
df1 = pd.read_csv('2016_commons.csv')
df2 = pd.read_csv('2017_commons.csv')
df = pd.concat([df1, df2])
df = df.drop(['Government'], axis=1)
df = df.loc[(df['Party'] == 'Conservative') | (df['Party'] == 'Labour')]
df = df.loc[df['Name'].isin(politicians)]
df['Date'] = pd.to_datetime(df['Date'])
mask = (df['Date'] > '2016-6-1') & (df['Date'] <= '2017-6-30')
df = df.loc[mask]
df['party2'] = df['Name'].apply(lambda x: politicians_party[x])
df['ref_dummy'] = df['Name'].apply(lambda x: politicians_ref[x])
# df = df.loc[df['Topic'] == 'European Union']
cols = df.columns
cols = ['date', 'speaker', 'name', 'Party', 'Topic', 'transcript', 'party2', 'ref_dummy']
df.columns = cols
df.head()

,date,speaker,name,Party,Topic,transcript,party2,ref_dummy
31481,2016-06-06,Sarah Champion (Rotherham) (Lab),Sarah Champion,Labour,Communities and families,1. What assessment his Department has made of...,0.0,1
31482,2016-06-06,The Secretary of State for Communities and Loc...,Greg Clark,Conservative,Communities and families,"Domestic abuse is a devastating crime, and we ...",0.0,0
31483,2016-06-06,Sarah Champion,Sarah Champion,Labour,Communities and families,The Secretary of State knows how devastating d...,0.0,1
31484,2016-06-06,Greg Clark,Greg Clark,Conservative,Communities and families,"Yes, it is important that we have specialist s...",0.0,0
31485,2016-06-06,Jon Trickett (Hemsworth) (Lab),Jon Trickett,Labour,Communities and families,I welcome what the Secretary of State has just...,0.0,1


In [225]:
len(df.loc[df.Topic == 'European Union'])

4809

In [226]:
def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return t
        
    return topic

In [227]:
df['Topic'] = df['Topic'].apply(lambda x: preprocess(x))

In [228]:
len(df.loc[df.Topic == 'European Union'])

5891